# Dataset overview

In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from utils import get_target_pivot, merge_rare, enumerate_float
from feature_engineering import generate_event_features

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.3f}'.format)

## 1. Dataset import

In [3]:
DATA_DIR = 'data'

In [4]:
USERS_FILE_NAME = 'user_profile.csv'
EVENTS_FILE_NAME = 'events.csv'

In [5]:
users_file_path = os.path.join(DATA_DIR, USERS_FILE_NAME)
events_file_path = os.path.join(DATA_DIR, EVENTS_FILE_NAME)

In [6]:
df = pd.read_csv(users_file_path).set_index('user_id').sort_index()

In [7]:
events_df = pd.read_csv(events_file_path)

In [8]:
df['reg_ts'] = pd.to_datetime(df['reg_ts'], unit='s')

In [9]:
events_df['event_ts'] = pd.to_datetime(events_df['event_ts'], unit='s')

In [10]:
df.rename(columns={'target': 'ltv_30'}, inplace=True)

In [11]:
df.shape

(4862, 4)

In [12]:
events_df.shape

(423862, 4)

In [13]:
df.head()

,reg_ts,country,entry_point,ltv_30
user_id,,,,
1,2025-01-01 02:06:17,US,android,1468.187
2,2025-01-01 08:06:23,PL,android,619.849
3,2025-01-01 13:07:50,US,android,97.804
4,2025-01-01 15:21:41,HU,android,71.662
5,2025-01-02 07:27:59,PL,android,0.000


In [14]:
events_df.head()

,event_ts,user_id,event_name,event_value
0,2025-01-01 04:41:50,843,payment,23.710
1,2025-01-01 05:23:58,2605,payment,24.451
2,2025-01-01 06:31:28,3715,payment,24.451
3,2025-01-01 06:41:23,3715,payment,24.451
4,2025-01-01 06:52:47,3715,payment,24.451


In [15]:
TARGET = 'ltv_30'

## 2. Users data overview

In [16]:
df[TARGET].sum()

np.float64(706138.8474897386)

In [17]:
df[TARGET].mean()

np.float64(145.2362911332247)

In [18]:
(df[TARGET] == 0).mean()

np.float64(0.637186343068696)

### 2.1. `entry_point`

In [19]:
get_target_pivot(df, col='entry_point', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
entry_point,,,,
web,892.33,57,1.2%,7.2%
ios,225.35,778,16.0%,24.8%
android,119.18,4 027,82.8%,68.0%


In [20]:
df['entry_point=android'] = (df['entry_point'] == 'android').astype(int)

In [21]:
get_target_pivot(df, col='entry_point=android', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
entry_point=android,,,,
0,270.88,835,17.2%,32.0%
1,119.18,4 027,82.8%,68.0%


### 2.2. `country`

In [22]:
df['country'].value_counts().head(10)

country
US    737
BR    262
FR    259
DE    252
IN    221
TR    218
PL    214
IT    179
GB    162
ES    150
Name: count, dtype: int64

In [23]:
df['country'] = merge_rare(df['country'])

In [24]:
get_target_pivot(df, col='country', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
country,,,,
CH,456.60,55,1.1%,3.6%
GB,340.63,162,3.3%,7.8%
US,290.42,737,15.2%,30.3%
DE,283.55,252,5.2%,10.1%
RO,231.55,63,1.3%,2.1%
CA,230.07,70,1.4%,2.3%
AU,175.74,73,1.5%,1.8%
PL,156.29,214,4.4%,4.7%
IT,156.09,179,3.7%,4.0%


In [25]:
country_rank = df[['country', 'ltv_30']].groupby(by='country').mean()['ltv_30'].sort_values(ascending=False).index.tolist()
# country_rank

In [26]:
country_ranks_dict = dict(zip(country_rank, range(1, len(country_rank) + 1)))
country_ranks_dict

{'CH': 1,
 'GB': 2,
 'US': 3,
 'DE': 4,
 'RO': 5,
 'CA': 6,
 'AU': 7,
 'PL': 8,
 'IT': 9,
 'ES': 10,
 'FR': 11,
 'ZA': 12,
 'OTHER': 13,
 'TR': 14,
 'BR': 15,
 'CL': 16,
 'AR': 17,
 'KZ': 18,
 'IN': 19,
 'PK': 20,
 'MX': 21}

In [27]:
df['country_e'] = df['country'].map(country_ranks_dict)

In [28]:
# get_target_pivot(df, col='country_e', target_col=TARGET)

## 3. Events data overview

In [29]:
events_df['event_name'].value_counts()

event_name
battle             179286
wealth_on_login     83677
login               83677
finish_quest        47922
level_up            24286
payment              5014
Name: count, dtype: int64

In [30]:
df = df.join(generate_event_features(df, events_df), how='left')

### 3.1. `battle`

In [31]:
df['battles_won_d0_q'] = enumerate_float(df['battles_won_d0'], q=5, process_zeros=True)
df['battles_won_d7_q'] = enumerate_float(df['battles_won_d7'], q=9, process_zeros=True)

In [32]:
get_target_pivot(df, col='battles_won_d0_q', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
battles_won_d0_q,,,,
0.000,65.66,2 787,57.3%,25.9%
0.999,130.27,667,13.7%,12.3%
2.000,119.79,228,4.7%,3.9%
4.000,217.29,364,7.5%,11.2%
12.000,247.84,406,8.4%,14.2%
30.000,559.05,410,8.4%,32.5%


In [33]:
get_target_pivot(df, col='battles_won_d7_q', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
battles_won_d7_q,,,,
0.000,1.79,2 186,45.0%,0.6%
0.999,3.33,298,6.1%,0.1%
1.222,13.99,329,6.8%,0.7%
4.000,45.51,284,5.8%,1.8%
12.000,90.42,293,6.0%,3.8%
22.000,175.23,294,6.0%,7.3%
35.000,207.82,288,5.9%,8.5%
55.000,288.74,302,6.2%,12.3%
79.000,395.66,292,6.0%,16.4%


### 3.2. `login`

In [34]:
df['session_time_d7_q'] = enumerate_float(df['session_time_d7'], q=10, process_zeros=True)

In [35]:
get_target_pivot(df, col='session_time_d7_q', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
session_time_d7_q,,,,
0.000,0.00,231,4.8%,0.0%
0.999,0.49,495,10.2%,0.0%
6.000,0.13,442,9.1%,0.0%
14.000,0.53,493,10.1%,0.0%
24.000,2.91,443,9.1%,0.2%
38.000,3.19,449,9.2%,0.2%
76.000,17.28,457,9.4%,1.1%
167.000,45.46,466,9.6%,3.0%
310.000,128.03,460,9.5%,8.3%


In [36]:
get_target_pivot(df, col='inactive_d1', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
inactive_d1,,,,
0,268.98,2 408,49.5%,91.7%
1,23.81,2 454,50.5%,8.3%


In [37]:
get_target_pivot(df, col='n_active_days', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
n_active_days,,,,
0,0.00,39,0.8%,0.0%
1,1.89,1 842,37.9%,0.5%
2,11.35,773,15.9%,1.2%
3,21.78,488,10.0%,1.5%
4,49.79,314,6.5%,2.2%
5,77.10,238,4.9%,2.6%
6,261.08,209,4.3%,7.7%
7,354.15,312,6.4%,15.6%
8,748.40,647,13.3%,68.6%


### 3.3. `wealth_on_login`

In [38]:
# get_target_pivot(df, col='wealth_on_login_max_d0', target_col=TARGET)

In [39]:
# get_target_pivot(df, col='wealth_on_login_max_d7', target_col=TARGET)

In [40]:
get_target_pivot(df, col='wealth_on_login_max_d0=802', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
wealth_on_login_max_d0=802,,,,
0,235.18,2 350,48.3%,78.3%
1,61.10,2 512,51.7%,21.7%


In [41]:
get_target_pivot(df, col='wealth_on_login_max_d7=802', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
wealth_on_login_max_d7=802,,,,
0,217.13,3 248,66.8%,99.9%
1,0.56,1 614,33.2%,0.1%


### 3.4. `finish_quest`

In [42]:
df['finish_quest_sum_d7_q'] = enumerate_float(df['finish_quest_sum_d7'], q=10, process_zeros=True)

In [43]:
get_target_pivot(df, col='finish_quest_sum_d7_q', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
finish_quest_sum_d7_q,,,,
0.000,4.32,2 723,56.0%,1.7%
9.999,58.90,274,5.6%,2.3%
30.000,51.59,186,3.8%,1.4%
70.000,49.93,194,4.0%,1.4%
120.000,122.65,218,4.5%,3.8%
180.000,142.37,208,4.3%,4.2%
260.000,165.53,203,4.2%,4.8%
378.000,154.48,215,4.4%,4.7%
570.000,271.86,216,4.4%,8.3%


### 3.5. `level_up`

In [44]:
get_target_pivot(df, col='level_up_max_d0', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
level_up_max_d0,,,,
0,65.33,2 306,47.4%,21.3%
2,58.09,320,6.6%,2.6%
3,109.40,766,15.8%,11.9%
4,74.63,225,4.6%,2.4%
5,166.45,177,3.6%,4.2%
6,387.00,49,1.0%,2.7%
7,207.96,91,1.9%,2.7%
8,169.25,139,2.9%,3.3%
9,236.80,400,8.2%,13.4%


In [45]:
get_target_pivot(df, col='level_up_max_d7', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
level_up_max_d7,,,,
0,1.08,1 761,36.2%,0.3%
2,1.19,280,5.8%,0.0%
3,2.14,543,11.2%,0.2%
4,5.71,171,3.5%,0.1%
5,4.86,89,1.8%,0.1%
6,3.08,28,0.6%,0.0%
7,5.14,62,1.3%,0.0%
8,7.16,84,1.7%,0.1%
9,28.85,297,6.1%,1.2%


### 3.6. `payment`

In [46]:
df['payment_sum_d0_q'] = enumerate_float(df['payment_sum_d0'], q=10, process_zeros=True)
df['payment_sum_d7_q'] = enumerate_float(df['payment_sum_d7'], q=10, process_zeros=True)

In [47]:
get_target_pivot(df, col='payment_sum_d0_q', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
payment_sum_d0_q,,,,
0.000,99.56,4 306,88.6%,60.7%
11.108,68.97,56,1.2%,0.5%
23.771,100.08,69,1.4%,1.0%
24.402,287.58,54,1.1%,2.2%
24.451,215.18,44,0.9%,1.3%
26.921,335.46,55,1.1%,2.6%
27.766,212.77,56,1.2%,1.7%
32.250,172.22,60,1.2%,1.5%
48.804,343.64,51,1.0%,2.5%


In [48]:
get_target_pivot(df, col='payment_sum_d7_q', target_col=TARGET)

,ltv_30,n_users,%_users,%_rev
payment_sum_d7_q,,,,
0.000,0.20,3 110,64.0%,0.1%
11.072,41.31,176,3.6%,1.0%
23.790,46.98,254,5.2%,1.7%
24.451,57.44,96,2.0%,0.8%
26.903,69.55,175,3.6%,1.7%
29.012,77.44,175,3.6%,1.9%
34.202,114.05,175,3.6%,2.8%
53.579,144.03,177,3.6%,3.6%
73.206,261.92,173,3.6%,6.4%


## 4. Dataset segmentation

In [49]:
train_df, dev_df = train_test_split(df, test_size=0.2, random_state=0)

In [50]:
print(train_df.shape)
print(dev_df.shape)

(3889, 44)
(973, 44)


In [51]:
df.loc[train_df.index, 'segment'] = 'TRAIN'
df.loc[dev_df.index, 'segment'] = 'DEV'

In [52]:
df['segment'].value_counts(dropna=False)

segment
TRAIN    3889
DEV       973
Name: count, dtype: int64

## 4. Dataset export

In [53]:
df.columns.tolist()

['reg_ts',
 'country',
 'entry_point',
 'ltv_30',
 'entry_point=android',
 'country_e',
 'battles_won_d0',
 'battles_won_d1',
 'battles_won_d3',
 'battles_won_d7',
 'battles_lost_d0',
 'battles_lost_d1',
 'battles_lost_d3',
 'battles_lost_d7',
 'session_time_d0',
 'session_time_d1',
 'session_time_d3',
 'session_time_d7',
 'inactive_d1',
 'n_active_days',
 'wealth_on_login_max_d0',
 'wealth_on_login_max_d1',
 'wealth_on_login_max_d3',
 'wealth_on_login_max_d7',
 'wealth_on_login_max_d0=802',
 'wealth_on_login_max_d7=802',
 'finish_quest_sum_d0',
 'finish_quest_sum_d1',
 'finish_quest_sum_d3',
 'finish_quest_sum_d7',
 'level_up_max_d0',
 'level_up_max_d1',
 'level_up_max_d3',
 'level_up_max_d7',
 'payment_sum_d0',
 'payment_sum_d1',
 'payment_sum_d3',
 'payment_sum_d7',
 'battles_won_d0_q',
 'battles_won_d7_q',
 'session_time_d7_q',
 'finish_quest_sum_d7_q',
 'payment_sum_d0_q',
 'payment_sum_d7_q',
 'segment']

In [54]:
FEATURES = [
    'entry_point=android',
    'country_e',
    'battles_won_d0',
    'battles_won_d1',
    'battles_won_d3',
    'battles_won_d7',
    'battles_lost_d0',
    'battles_lost_d1',
    'battles_lost_d3',
    'battles_lost_d7',
    'session_time_d0',
    'session_time_d1',
    'session_time_d3',
    'session_time_d7',
    'inactive_d1',
    'n_active_days',
    'wealth_on_login_max_d0',
    'wealth_on_login_max_d1',
    'wealth_on_login_max_d3',
    'wealth_on_login_max_d7',
    'finish_quest_sum_d0',
    'finish_quest_sum_d1',
    'finish_quest_sum_d3',
    'finish_quest_sum_d7',
    'level_up_max_d0',
    'level_up_max_d1',
    'level_up_max_d3',
    'level_up_max_d7',
    'payment_sum_d0',
    'payment_sum_d1',
    'payment_sum_d3',
    'payment_sum_d7',
    'wealth_on_login_max_d0=802',
    'wealth_on_login_max_d7=802',
]

In [55]:
EXPORT_COLS = FEATURES + ['segment', TARGET]

In [56]:
EXPORT_FILE_NAME = 'train_dev.csv'

In [57]:
export_file_path = os.path.join(DATA_DIR, EXPORT_FILE_NAME)

In [58]:
df[EXPORT_COLS].to_csv(export_file_path)